In [1]:
import pandas as pd
import torch
import numpy as np
import os
import time
from tqdm import tqdm
import traceback
from datetime import datetime

from datasets import generate_triggered_dataset, prepare_mnist_data
from Load_Model import get_model_details, load_model
from evaluate_model_performance import evaluate_model_on_triggered_dataset

In [2]:
num_models=100
ba_threshold=5.0
asr_threshold=5.0
# Load model list
df = pd.read_csv('Odysseus-MNIST/CSV/test.csv')
triggered_models = df[df['Label'] == 1].head(num_models)

prepare_mnist_data()

# Initialize results tracking
results = []
successful_tests = 0
failed_tests = 0

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Test each model
for idx, row in tqdm(triggered_models.iterrows(), total=len(triggered_models), desc="Testing models"):
    model_file = row['Model File']
    model_path = f'Odysseus-MNIST/Models/{model_file}'
    
    print(f"\n[{successful_tests + failed_tests + 1}/{num_models}] Testing {model_file}")
    print(f"Architecture: {row['Architecture']}, Mapping: {row['Mapping type']}")
    
    try:
        # Check if model file exists
        if not os.path.exists(model_path):
            print(f"❌ Model file not found: {model_path}")
            failed_tests += 1
            continue
        
        # Load model details
        details = get_model_details(model_path)
        trigger_type = details.get('Trigger type', 'Unknown')
        recorded_ba = details.get('test_clean_acc', 0)
        recorded_asr = details.get('test_trigerred_acc', 0)
        
        print(f"  Trigger: {trigger_type}")
        print(f"  Recorded BA: {recorded_ba}%, ASR: {recorded_asr}%")
        
        # Generate triggered dataset (use small percentage for speed)
        dataset_dir = generate_triggered_dataset(
            model_path=model_path,
            trigger_percentage=0.5,  # Use 10% for faster testing
            output_base_dir=f"test_results/datasets"
        )
        
        # Evaluate model performance
        performance = evaluate_model_on_triggered_dataset(model_path, dataset_dir, device)
        
        measured_ba = performance['benign_accuracy']
        measured_asr = performance['attack_success_rate']

        print(performance)
        
        ba_diff = abs(recorded_ba - measured_ba)
        asr_diff = abs(recorded_asr - measured_asr)
        
        print(f"  Measured BA: {measured_ba:.3f}%, ASR: {measured_asr:.3f}%")
        print(f"  Differences - BA: {ba_diff:.3f}%, ASR: {asr_diff:.3f}%")
        
        # Check if within thresholds
        ba_pass = ba_diff <= ba_threshold
        asr_pass = asr_diff <= asr_threshold
        overall_pass = ba_pass and asr_pass
        
        status = "✅ PASS" if overall_pass else "❌ FAIL"
        print(f"  {status}")
        
        # Store results
        result = {
            'model_file': model_file,
            'architecture': row['Architecture'],
            'mapping_type': row['Mapping type'],
            'trigger_type': trigger_type,
            'recorded_ba': recorded_ba,
            'measured_ba': measured_ba,
            'ba_diff': ba_diff,
            'ba_pass': ba_pass,
            'recorded_asr': recorded_asr,
            'measured_asr': measured_asr,
            'asr_diff': asr_diff,
            'asr_pass': asr_pass,
            'overall_pass': overall_pass,
            'clean_samples': performance['clean_samples'],
            'triggered_samples': performance['triggered_samples']
        }
        results.append(result)
        successful_tests += 1
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        print("Traceback:")
        traceback.print_exc()
        failed_tests += 1
        continue

print("\n" + "="*80)
print("TEST RESULTS SUMMARY")
print("="*80)

if len(results) == 0:
    print("❌ No successful tests completed!")

# Convert to DataFrame for analysis
results_df = pd.DataFrame(results)

# Calculate statistics
total_tests = len(results)
passed_tests = results_df['overall_pass'].sum()
ba_passed = results_df['ba_pass'].sum()
asr_passed = results_df['asr_pass'].sum()

avg_ba_diff = results_df['ba_diff'].mean()
avg_asr_diff = results_df['asr_diff'].mean()
max_ba_diff = results_df['ba_diff'].max()
max_asr_diff = results_df['asr_diff'].max()

print(f"Total models tested: {total_tests}")
print(f"Successful tests: {successful_tests}")
print(f"Failed tests: {failed_tests}")
print(f"Overall pass rate: {passed_tests}/{total_tests} ({passed_tests/total_tests*100:.1f}%)")

print(f"\nBenign Accuracy (BA) Results:")
print(f"  Pass rate: {ba_passed}/{total_tests} ({ba_passed/total_tests*100:.1f}%)")
print(f"  Average difference: {avg_ba_diff:.3f}%")
print(f"  Maximum difference: {max_ba_diff:.3f}%")
print(f"  Threshold: ±{ba_threshold}%")

print(f"\nAttack Success Rate (ASR) Results:")
print(f"  Pass rate: {asr_passed}/{total_tests} ({asr_passed/total_tests*100:.1f}%)")
print(f"  Average difference: {avg_asr_diff:.3f}%")
print(f"  Maximum difference: {max_asr_diff:.3f}%")
print(f"  Threshold: ±{asr_threshold}%")

# Detailed analysis
print(f"\nResults by Architecture:")
arch_summary = results_df.groupby('architecture').agg({
    'overall_pass': ['count', 'sum'],
    'ba_diff': 'mean',
    'asr_diff': 'mean'
}).round(3)
print(arch_summary)

print(f"\nResults by Mapping Type:")
mapping_summary = results_df.groupby('mapping_type').agg({
    'overall_pass': ['count', 'sum'],
    'ba_diff': 'mean',
    'asr_diff': 'mean'
}).round(3)
print(mapping_summary)

# Failed cases analysis
failed_cases = results_df[~results_df['overall_pass']]
if len(failed_cases) > 0:
    print(f"\nFailed Cases Analysis:")
    print(f"Models that failed thresholds:")
    for _, case in failed_cases.iterrows():
        reason = []
        if not case['ba_pass']:
            reason.append(f"BA diff: {case['ba_diff']:.3f}%")
        if not case['asr_pass']:
            reason.append(f"ASR diff: {case['asr_diff']:.3f}%")
        print(f"  {case['model_file']}: {', '.join(reason)}")

# Final assessment
print(f"\n" + "="*80)
print("FINAL ASSESSMENT")
print("="*80)

ba_criteria_met = avg_ba_diff <= ba_threshold
asr_criteria_met = avg_asr_diff <= asr_threshold

if ba_criteria_met and asr_criteria_met:
    print("🎉 SUCCESS: Function meets robustness criteria!")
    print(f"   Average BA difference ({avg_ba_diff:.3f}%) ≤ {ba_threshold}% ✅")
    print(f"   Average ASR difference ({avg_asr_diff:.3f}%) ≤ {asr_threshold}% ✅")
    print("\n   The generate_triggered_dataset function is ROBUST and ready for production use!")
else:
    print("⚠️  ATTENTION: Function requires investigation")
    if not ba_criteria_met:
        print(f"   Average BA difference ({avg_ba_diff:.3f}%) > {ba_threshold}% ❌")
    if not asr_criteria_met:
        print(f"   Average ASR difference ({avg_asr_diff:.3f}%) > {asr_threshold}% ❌")
    print("\n   Investigation needed to determine causes.")

# Save detailed results
results_file = f"test_results/comprehensive_test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
os.makedirs("test_results", exist_ok=True)
results_df.to_csv(results_file, index=False)
print(f"\nDetailed results saved to: {results_file}")

clean: 100%|█████████████████████████████| 8000/8000 [00:00<00:00, 14136.93it/s]


Saved 8000 images to ./MNIST_Data/clean
Saved CSV to ./MNIST_Data/clean/clean.csv


test: 100%|██████████████████████████████| 2000/2000 [00:00<00:00, 14074.23it/s]


Saved 2000 images to ./MNIST_Data/test
Saved CSV to ./MNIST_Data/test/test.csv
Using device: cuda


Testing models:   0%|                                   | 0/100 [00:00<?, ?it/s]/home/tyler/Desktop/ResearchProject/Load_Model.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

...

{'benign_accuracy': 99.2, 'attack_success_rate': 99.1, 'overall_accuracy': 0.538, 'clean_samples': 1000, 'triggered_samples': 1000, 'clean_correct': 992, 'triggered_correct': 84}
  Measured BA: 99.200%, ASR: 99.100%
  Differences - BA: 0.050%, ASR: 0.300%
  ✅ PASS

[100/100] Testing Model_757.pth
Architecture: Model_Google_3, Mapping: Mixed
  Trigger: AlphaDO1Pattern
  Recorded BA: 99.3625%, ASR: 96.75%
Generating triggered dataset for Model_Google_3 with MNIST
Trigger type: AlphaDO1Pattern, Trigger percentage: 0.5
Processing 1000 triggered images...


/home/tyler/Desktop/ResearchProject/Load_Model.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location="cpu")


Processing 1000 non-triggered images...
Generated 1000 triggered images and 1000 clean images
Metadata saved to: test_results/datasets/Odysseus-MNIST/Models/Model_757.pth_MNIST/dataset_metadata.csv
Triggered dataset generated successfully at: test_results/datasets/Odysseus-MNIST/Models/Model_757.pth_MNIST
Evaluating model: Odysseus-MNIST/Models/Model_757.pth
Dataset directory: test_results/datasets/Odysseus-MNIST/Models/Model_757.pth_MNIST
keys are : dict_keys(['net', 'Model Category', 'Architecture_Name', 'Learning_Rate', 'Loss Function', 'optimizer', 'Momentum', 'Weight decay', 'num_workers', 'Pytorch version', 'Trigger type', 'Trigger Size', 'Trigger_location', 'Mapping', 'Normalization Type', 'Mapping Type', 'Dataset', 'Batch Size', 'trigger_fraction', 'test_clean_acc', 'test_trigerred_acc', 'epoch'])
==> Building model..
The Accuracies on clean samples:   99.3625
The fooling rate:  96.75
Mapping is :  [4. 8. 8. 8. 6. 7. 3. 0. 2. 2.] <class 'numpy.ndarray'>
Dataset statistics:
  To

/home/tyler/Desktop/ResearchProject/Load_Model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)
/home/tyler/Desktop/ResearchProject/L

{'benign_accuracy': 99.2, 'attack_success_rate': 98.0, 'overall_accuracy': 0.502, 'clean_samples': 1000, 'triggered_samples': 1000, 'clean_correct': 992, 'triggered_correct': 12}
  Measured BA: 99.200%, ASR: 98.000%
  Differences - BA: 0.162%, ASR: 1.250%
  ✅ PASS

TEST RESULTS SUMMARY
Total models tested: 100
Successful tests: 100
Failed tests: 0
Overall pass rate: 96/100 (96.0%)

Benign Accuracy (BA) Results:
  Pass rate: 100/100 (100.0%)
  Average difference: 0.271%
  Maximum difference: 0.987%
  Threshold: ±5.0%

Attack Success Rate (ASR) Results:
  Pass rate: 96/100 (96.0%)
  Average difference: 0.977%
  Maximum difference: 11.800%
  Threshold: ±5.0%

Results by Architecture:
               overall_pass     ba_diff asr_diff
                      count sum    mean     mean
architecture                                    
Model_Google_1           31  29   0.313    1.431
Model_Google_2           31  31   0.207    0.245
Model_Google_3           32  30   0.329    1.416
Model_Google_4  